<a href="https://colab.research.google.com/github/VaishnavSharan/2000320120187/blob/main/first%20question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, jsonify, request
import requests
import gevent
from gevent import monkey

monkey.patch_all()

app = Flask(__name__)

def fetch_numbers(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data.get('numbers', [])
    except:
        pass
    return []

@app.route('/numbers')
def get_numbers():
    urls = request.args.getlist('url')
    urls = list(set(urls))  # Remove duplicate URLs if any

    jobs = [gevent.spawn(fetch_numbers, url) for url in urls]
    gevent.joinall(jobs, timeout=0.5)  # Set the timeout to 0.5 seconds

    numbers = []
    for job in jobs:
        result = job.value
        if result:
            numbers.extend(result)

    numbers = sorted(list(set(numbers)))  # Sort and remove duplicates

    return jsonify({"numbers": numbers})

if __name__ == '__main__':
    app.run(host='localhost', port=8008)